In [3]:
# Import necessary libraries
from clearml import Task
from pymongo import MongoClient
import requests
import json
from bs4 import BeautifulSoup

# Initialize ClearML Task for tracking the ETL pipeline
task = Task.init(project_name="RAG System", task_name="ETL Pipeline")

# Connect to MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["rag_system"]
collection = db["raw_data"]

# Function to ingest ROS2 documentation from GitHub
def ingest_github_docs(repo_url):
    """Fetch GitHub repo README or documentation."""
    response = requests.get(repo_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        text_content = soup.get_text()
        # Save data to MongoDB
        collection.insert_one({
            "source": "GitHub",
            "url": repo_url,
            "content": text_content
        })
        print(f"Ingested data from {repo_url}")
    else:
        print(f"Failed to fetch {repo_url}: {response.status_code}")

# Function to ingest YouTube video descriptions
def ingest_youtube_videos(api_key, channel_id):
    """Fetch video titles and descriptions from a YouTube channel."""
    youtube_api_url = f"https://www.googleapis.com/youtube/v3/search?channelId={channel_id}&part=snippet&type=video&maxResults=10&key={api_key}"
    response = requests.get(youtube_api_url)
    if response.status_code == 200:
        videos = response.json()["items"]
        for video in videos:
            video_data = {
                "source": "YouTube",
                "url": f"https://www.youtube.com/watch?v={video['id']['videoId']}",
                "title": video["snippet"]["title"],
                "description": video["snippet"]["description"]
            }
            # Save data to MongoDB
            collection.insert_one(video_data)
            print(f"Ingested video: {video_data['title']}")
    else:
        print(f"Failed to fetch YouTube data: {response.status_code}")

# ClearML Task: Track data ingestion
task.get_logger().report_text("Starting ETL pipeline")

# Ingest data from GitHub
ros2_repo_url = "https://github.com/ros2/ros2_documentation"
ingest_github_docs(ros2_repo_url)

"""
# Ingest data from YouTube
youtube_api_key = "your_youtube_api_key"
ros2_channel_id = "your_ros2_channel_id"
ingest_youtube_videos(youtube_api_key, ros2_channel_id)
"""

# Print URLs from MongoDB
def print_ingested_urls():
    """Query MongoDB to print all ingested URLs."""
    documents = collection.find({}, {"url": 1, "_id": 0})
    urls = [doc["url"] for doc in documents]
    print("Ingested URLs:")
    print("\n".join(urls))

# Display URLs in the notebook
print_ingested_urls()

# ClearML Task: Mark pipeline as complete
task.close()

Retrying (Retry(total=239, connect=239, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9414db9480>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /auth.login
Retrying (Retry(total=238, connect=238, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f946c3138e0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /auth.login
Retrying (Retry(total=237, connect=237, read=240, redirect=240, status=240)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f9414db9ab0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution')': /auth.login


KeyboardInterrupt: 